# LAB SINGULARITY - DENGUE

## IMPORTS

In [1]:
import pandas as pd
import numpy as np
import json as js
import datetime


## COLUMNAS DE INTERES

In [2]:
cols = ["PAC_HOS", "SEXO", "Municipio_ocurrencia", "CON_FIN", "FEC_NOT", "ANO","EDAD", "Departamento_ocurrencia"] 
#datos = pd.read_excel("Dengue_todos.xlsx", usecols=cols)

## CLEAN DATOS

In [4]:
#datos.to_csv("df_clean.csv" , index = False, sep = ",", encoding='utf-8')
df = pd.read_csv("df_clean.csv")
clean_df = df.drop(index = df.index[720714:720747])
#clean_df[clean_df.isnull().any(axis=1)]
##clean_df["Departamento_ocurrencia"].unique()[:-1]


## CLASE NODO

In [5]:
class Nodo_Dep:
    def __init__(self, dpto_index, dpto_name,\
        next = None,prev = None):  

        self.dpto_index = dpto_index  # department index  
        self.dpto_name = dpto_name    # department name
        #self.info = clean_df[clean_df[nom].str.contains(info, na = False)]
        self.dpto_info = clean_df[clean_df["Departamento_ocurrencia"] == dpto_name]      # dataframe with info regarding the departement
        #self.model = model
        self.next = next
        self.prev = prev

## CLASE DOUBLE LINKED LIST

In [21]:
class DoubleLinkedList:
    def __init__(self):
            self.head = None
            self.tail = None

    def AddNode(self,dpto_index, dpto_name):
        P = Nodo_Dep(dpto_index=dpto_index, dpto_name=dpto_name)
        if (self.head == None):
            self.head = P
            self.tail = P 
        else:
            self.tail.next = P
            P.prev = self.tail
            self.tail = P  
        self.tail.next = self.head
        self.head.prev = self.tail
        
    def Recorrido(self):
        P = self.head
        print(P.dpto_name , P.dpto_info)
        #print(P.dpto_name, P.dpto_info["Departamento_ocurrencia"].unique()) CONFIRMAR QUE DPTO NAME == DATO OCUR
        P = P.prev
        while(P != self.head):
            print(P.dpto_name, P.dpto_info)
            #print(P.dpto_name, P.dpto_info["Departamento_ocurrencia"].unique())
            P = P.prev


    def NumHospitalizados(self,dpto_name):
        dpto_info = clean_df[clean_df["Departamento_ocurrencia"] ==dpto_name ]
        dpto_search=dpto_info[dpto_info["PAC_HOS"] == 1] # selecciona los pacientes Hospitalizados en el dpto de interes, (1): hospitalizado
        #muertos = len(clean_df[dpto_info["PAC_HOS"] == 1].axes[0])
        Pac_Hos=len(dpto_search)
        print(Pac_Hos)



    def Search(self, dptoName):
        P = self.head
        Q = self.tail
        sw=False
        sw1= False
        while(sw == False and sw1== False ):
            if (P.dpto_name==dptoName or Q.dpto_name==dptoName):        
                print(dptoName)
                return dptoName 
            elif (P==Q or P.next==Q or Q.prev == P ):
                return False
            else:
                P = P.next
                Q = Q.prev

    def num_hospitalizados(self,dptoName):
        
        dpto_info = clean_df[clean_df["Departamento_ocurrencia"] == dptoName]
        dpto_search=dpto_info[dpto_info["PAC_HOS"] == 1] # selecciona los pacientes Hospitalizados en el dpto de interes, (1): hospitalizado
        #muertos = len(clean_df[dpto_info["PAC_HOS"] == 1].axes[0])
        Pac_Hos=len(dpto_search)
        print("HOSPITALIZADOS: ", Pac_Hos)

    def num_sexo(self,dptoName):
        dpto_info = clean_df[clean_df["Departamento_ocurrencia"] == dptoName]
        dpto_searchM=dpto_info[dpto_info["SEXO"] == "M"] # selecciona l"masculinos" en el dpto de interes, (1): hospitalizado
        SEXOM=len(dpto_searchM.axes[0])
        dpto_searchF=dpto_info[dpto_info["SEXO"] == "F"] # selecciona "femeninos" en el dpto de interes
        SEXOF=len(dpto_searchF.axes[0])
        print("MUJERES: ",  SEXOF)
        print("HOMBRES: ",SEXOM)

    def mes_casos(self, dptoName):
        dpto_interes = clean_df[clean_df["Departamento_ocurrencia"] == dptoName]
        fecha = dpto_interes.groupby("FEC_NOT").value_counts().reset_index()
        #Con_Final = dpto_interes.groupby("Municipio_ocurrencia")[["FEC_NOT"]].value_counts().reset_index()
        fecha['MES'] = pd.to_datetime(fecha["FEC_NOT"]).dt.month
        FLista= fecha['MES'].value_counts().reset_index().values.tolist()[0]
        #Con_Final['MES'].value_counts()
        #fecha['MES'].strftime(%m)
        print("Mes:", FLista[0], "Registro mayor numero de casos")

    def dia_casos(self, dptoName):
        dpto_interes = clean_df[clean_df["Departamento_ocurrencia"] == dptoName]
        fecha = dpto_interes.groupby("FEC_NOT").value_counts().reset_index()
        #Con_Final = dpto_interes.groupby("Municipio_ocurrencia")[["FEC_NOT"]].value_counts().reset_index()
        fecha['DIA'] = pd.to_datetime(fecha["FEC_NOT"]).dt.day
        FLista= fecha['DIA'].value_counts().reset_index().values.tolist()[0]
        #Con_Final['MES'].value_counts()
        #fecha['MES'].strftime(%m)
        print("Dia:", FLista[0], "Registro mayor numero de casos")

    def casos_municipio(self, dptoName):
        dpto_interes = clean_df[clean_df["Departamento_ocurrencia"] == dptoName]#Dpto de interes
        MasCasos_list = dpto_interes['Municipio_ocurrencia'].value_counts().reset_index().values.tolist()[0] #lista de municipio con mas registros
        print("el municipio con mas registrados es:", MasCasos_list[0], "con ", MasCasos_list[1], "casos")

    def muertos_municipio(self, dptoName):
        dpto_interes= clean_df[clean_df["Departamento_ocurrencia"] == dptoName] #Dpto de interes
        Con_Final = dpto_interes.groupby("Municipio_ocurrencia")[["CON_FIN"]].value_counts().reset_index() #agrupa municipio por columan de condicion final
        MasMuertos_list =Con_Final[Con_Final["CON_FIN"] == 2].sort_values(by=0, ascending=False).reset_index().values.tolist()[0]
        #lista desendente de casos de muerte :2, [0]: pioscion con mas muertos
        print("El municipio mas muertos registrados es:", MasMuertos_list[1], ".CANT:", MasMuertos_list[3] )
    


## PRUEBA METODOS

### MUNICIPIOS

#### MAYOR NUMERO DE CASOS

In [9]:
dpto_interes = clean_df[clean_df["Departamento_ocurrencia"] == "ATLANTICO"]#Dpto de interes
MasCasos_list=dpto_interes['Municipio_ocurrencia'].value_counts().reset_index().values.tolist()[0] #lista de municipio con mas registros

print("el municipio con mas registrados es:", MasCasos_list[0], "con ", MasCasos_list[1], "casos")
MasCasos_list

el municipio con mas registrados es: BARRANQUILLA con  14971 casos


['BARRANQUILLA', 14971]

#### MAYOR NUMERO DE MUERTOS

In [10]:
dpto_interes= clean_df[clean_df["Departamento_ocurrencia"] == "SANTANDER"] #Dpto de interes
Con_Final = dpto_interes.groupby("Municipio_ocurrencia")[["CON_FIN"]].value_counts().reset_index() #agrupa municipio por columan de condicion final
MasMuertos_list =Con_Final[Con_Final["CON_FIN"] == 2].sort_values(by=0, ascending=False).reset_index().values.tolist()[0]
 #lista desendente de casos de muerte :2, [0]: pioscion con mas muertos
print("El municipio mas muertos registrados es:", MasMuertos_list[1], ".CANT:", MasMuertos_list[3] )


El municipio mas muertos registrados es: * SANTANDER. MUNICIPIO DESCONOCIDO .CANT: 1


### DEPARTAMENTO

#### NUMERO DE HOSPITALIZADOS

In [11]:
dpto_info = clean_df[clean_df["Departamento_ocurrencia"] == "ATLANTICO"]
dpto_search=dpto_info[dpto_info["PAC_HOS"] == 1] # selecciona los pacientes Hospitalizados en el dpto de interes, (1): hospitalizado
#muertos = len(clean_df[dpto_info["PAC_HOS"] == 1].axes[0])
Pac_Hos=len(dpto_search)
print(Pac_Hos)

#df_list = dpto_search.values.tolist()
#print(len(df_list))
#print(df_list)

12333


#### CANTIDAD DE CASOS POR SEXO

In [12]:
dpto_info = clean_df[clean_df["Departamento_ocurrencia"] == "VICHADA"]
dpto_searchM=dpto_info[dpto_info["SEXO"] == "M"] # selecciona l"masculinos" en el dpto de interes, (1): hospitalizado
SEXOM=len(dpto_searchM.axes[0])
dpto_searchF=dpto_info[dpto_info["SEXO"] == "F"] # selecciona "femeninos" en el dpto de interes
SEXOF=len(dpto_searchF.axes[0])
print("numero de mujeres: ", SEXOF)
print("numero de hombres: ",SEXOM)
list = [SEXOM, SEXOF]


numero de mujeres:  749
numero de hombres:  868


### TEMPORALES

#### MES CON MAS CASOS

In [14]:
dpto_interes = clean_df[clean_df["Departamento_ocurrencia"] == "ATLANTICO"]
fecha = dpto_interes.groupby("FEC_NOT").value_counts().reset_index()
#Con_Final = dpto_interes.groupby("Municipio_ocurrencia")[["FEC_NOT"]].value_counts().reset_index()
fecha['MES'] = pd.to_datetime(fecha["FEC_NOT"]).dt.month
FLista= fecha['MES'].value_counts().reset_index().values.tolist()[0]
#Con_Final['MES'].value_counts()
#fecha['MES'].strftime(%m)

print("Mes:", FLista[0], "Registro mayor numero de casos")

Mes: 1 Registro mayor numero de casos


#### DIA CON MAS CASOS

In [16]:

dpto_interes = clean_df[clean_df["Departamento_ocurrencia"] == "ATLANTICO"]
fecha = dpto_interes.groupby("FEC_NOT").value_counts().reset_index()
#Con_Final = dpto_interes.groupby("Municipio_ocurrencia")[["FEC_NOT"]].value_counts().reset_index()
fecha['DIA'] = pd.to_datetime(fecha["FEC_NOT"]).dt.day
FLista= fecha['DIA'].value_counts().reset_index().values.tolist()[0]
#Con_Final['MES'].value_counts()
#fecha['MES'].strftime(%m)

print("Dia:", FLista[0], "Registro mayor numero de casos")

Dia: 4 Registro mayor numero de casos


### CODE TO RUN

In [22]:
lista_d = DoubleLinkedList()
for i,dpto in enumerate(clean_df["Departamento_ocurrencia"].unique()[:-1]): # ["Departamento_ocurrencia"].unique()[:-1] : hasta la penultima 
    #print(dpto)
    lista_d.AddNode(i, dpto) #i : dpto index , dpto: dpto name
    print("- - - -")
    lista_d.num_sexo(lista_d.Search(dpto))
    lista_d.num_hospitalizados(lista_d.Search(dpto))
    lista_d.mes_casos(lista_d.Search(dpto))
    lista_d.dia_casos(lista_d.Search(dpto))


- - - -
SANTANDER
MUJERES:  35964
HOMBRES:  39828
SANTANDER
HOSPITALIZADOS:  17976
SANTANDER
Mes: 5 Registro mayor numero de casos
SANTANDER
Dia: 18 Registro mayor numero de casos
- - - -
CORDOBA
MUJERES:  6353
HOMBRES:  7588
CORDOBA
HOSPITALIZADOS:  6682
CORDOBA
Mes: 1 Registro mayor numero de casos
CORDOBA
Dia: 22 Registro mayor numero de casos
- - - -
QUINDIO
MUJERES:  11735
HOMBRES:  12376
QUINDIO
HOSPITALIZADOS:  6051
QUINDIO
Mes: 3 Registro mayor numero de casos
QUINDIO
Dia: 25 Registro mayor numero de casos
- - - -
HUILA
MUJERES:  20370
HOMBRES:  22116
HUILA
HOSPITALIZADOS:  15936
HUILA
Mes: 1 Registro mayor numero de casos
HUILA
Dia: 4 Registro mayor numero de casos
- - - -
META
MUJERES:  20073
HOMBRES:  23405
META
HOSPITALIZADOS:  13810
META
Mes: 3 Registro mayor numero de casos
META
Dia: 9 Registro mayor numero de casos
- - - -
VICHADA
MUJERES:  749
HOMBRES:  868
VICHADA
HOSPITALIZADOS:  539
VICHADA
Mes: 4 Registro mayor numero de casos
VICHADA
Dia: 5 Registro mayor numero de

###